In [467]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## import the required Libraries




In [468]:
import os
import numpy as np
from sklearn.model_selection import train_test_split


from keras.models import Model
from keras.layers import Input, Embedding, SpatialDropout1D, Dropout, Conv1D, MaxPool1D, Flatten, concatenate, Dense, \
    LSTM, Bidirectional, Activation, MaxPooling1D, Add, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D, RepeatVector, \
    TimeDistributed, Permute, multiply, Lambda, add, Masking, BatchNormalization, Softmax, Reshape, ReLU, \
    ZeroPadding1D, subtract
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import tensorflow.keras.backend as K
import tensorflow as tf
from keras import backend as K, initializers, regularizers, constraints


# Import our dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from tensorflow.keras.layers import Layer, InputSpec
import numpy as np

from statistics import mode

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
%matplotlib inline
import tensorflow as tf


# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.metrics import Metric
from tensorflow.python.keras.utils import metrics_utils
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.keras.utils.generic_utils import to_list
import tensorflow_hub as hub



import re
import nltk
from nltk.corpus import stopwords

# neural
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential

#Metrics
from sklearn.metrics import balanced_accuracy_score

from sklearn.feature_extraction.text import TfidfVectorizer
from mlxtend.plotting import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier

##LOAD THE MAMS DATASET


In [469]:
df2_train = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_MAMS/ATSA_train_single.csv", encoding='utf8')

In [470]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_MAMS/ATSA_test_single.csv", encoding='utf8')

In [471]:
df2_val = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_MAMS/ATSA_val_single.csv", encoding='utf8')

In [472]:
df2_val

,sentence,aspect_terms,polarity
0,"After a couple of drinks, the apps--I like the...",drinks,neutral
1,"After a couple of drinks, the apps--I like the...",roll,positive
2,"After a couple of drinks, the apps--I like the...",cripsy squid,positive
3,The basil pepper mojito was a little daunting ...,basil pepper mojito,negative
4,The basil pepper mojito was a little daunting ...,flavor,positive
...,...,...,...
1327,The Food The menu is better suited to the snac...,bar,neutral
1328,Another favorite is the hanger steak which is ...,hanger steak,positive
1329,Another favorite is the hanger steak which is ...,amount of arugula,positive
1330,Another favorite is the hanger steak which is ...,a balsamic vinegar reduction served,neutral


##PREPROCESSING

In [473]:
## change the labels to integers for the training phase
df2_train['polarity'][df2_train["polarity"] == 'negative'] = 0
df2_train['polarity'][df2_train["polarity"] == 'positive'] = 1
df2_train['polarity'][df2_train["polarity"] == 'neutral'] = 2




In [474]:
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1
df2_test['polarity'][df2_test["polarity"] == 'neutral'] = 2



In [475]:
df2_val['polarity'][df2_val["polarity"] == 'negative'] = 0
df2_val['polarity'][df2_val["polarity"] == 'positive'] = 1
df2_val['polarity'][df2_val["polarity"] == 'neutral'] = 2


In [476]:
train_df = df2_train

In [477]:
## Convert polarity Values to Integers
train_df["polarity"] = pd.to_numeric(train_df["polarity"],errors='coerce')


In [478]:
## Split the Dataset into Training and Testing Datasets
#X_train, y_train= train_test_split(train_df,train_df['polarity'],stratify=train_df['polarity'])

X_train  = train_df
y_train = train_df['polarity']

In [479]:
X_test = df2_test

In [480]:
## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [481]:
y_test = X_test["polarity"]

In [482]:
X_val = df2_val

In [483]:
## Convert polarity Values to Integers
X_val["polarity"] = pd.to_numeric(X_val["polarity"],errors='coerce')


In [484]:
y_val = X_val["polarity"]

In [485]:
NUM_WORDS = 100000 ## MAx of words to keep, based on word frequency.
EMBEDDING_SIZE = 128 ## the length of the Vector the will

In [486]:
df2_test.aspect_terms.dtype

dtype('O')

## Embedding code


In [487]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_train_tokenized = tokenizer.texts_to_sequences(X_train.sentence) # list of tokenized sentences
Aspect_X_train_tokenized = tokenizer.texts_to_sequences(X_train['aspect_terms'].astype(str)) # list of tokenized sentences
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_terms']) # list of tokenized sentences
text_X_val_tokenized = tokenizer.texts_to_sequences(X_val.sentence) # list of tokenized sentences
Aspect_X_val_tokenized = tokenizer.texts_to_sequences(X_val['aspect_terms'].astype(str)) # list of tokenized sentences



In [488]:
##Maximum length of sentences
Max_Len =  max([len(one_title) for one_title in text_X_train_tokenized])
Max_Len

67

In [489]:
## Pads sequences(or sentences) to the same length.


text_X_train_padded = pad_sequences(text_X_train_tokenized, maxlen=Max_Len)
aspect_X_train_padded = pad_sequences(Aspect_X_train_tokenized, maxlen=1)

text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)

text_X_val_padded = pad_sequences(text_X_val_tokenized, maxlen=Max_Len)
aspect_X_val_padded = pad_sequences(Aspect_X_val_tokenized, maxlen=1)


In [490]:
## Concatenate the Embedding presentation of the sentences and aspects to Create training and testing dataset
train_data =  [text_X_train_padded,aspect_X_train_padded]
test_data =   [text_X_test_padded,aspect_X_test_padded]
val_data = [text_X_val_padded,aspect_X_val_padded]

In [491]:

# modified based on `https://gist.github.com/cbaziotis/7ef97ccf71cbc14366835198c09809d2`
class Attention(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
 e: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self, W_regularizer=None, u_regularizer=None, b_regularizer=None, W_constraint=None,
                 u_constraint=None, b_constraint=None, use_W=True, use_bias=False, return_self_attend=False,
                 return_attend_weight=True, **kwargs):
        self.supports_masking = True

        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.use_W = use_W
        self.use_bias = use_bias
        self.return_self_attend = return_self_attend    # whether perform self attention and return it
        self.return_attend_weight = return_attend_weight    # whether return attention weight
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        if self.use_W:
            self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),  initializer=self.init,
                                     name='{}_W'.format(self.name), regularizer=self.W_regularizer,
                                     constraint=self.W_constraint)
        if self.use_bias:
            self.b = self.add_weight(shape=(input_shape[1],), initializer='zero', name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer, constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],), initializer=self.init, name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer, constraint=self.u_constraint)

        super(Attention, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        if self.use_W:
            x = K.tanh(K.dot(x, self.W))

        ait = Attention.dot_product(x, self.u)
        if self.use_bias:
            ait += self.b

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        if self.return_self_attend:
            attend_output = K.sum(x * K.expand_dims(a), axis=1)
            if self.return_attend_weight:
                return [attend_output, a]
            else:
                return attend_output
        else:
            return a

    def compute_output_shape(self, input_shape):
        if self.return_self_attend:
            if self.return_attend_weight:
                return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[1])]
            else:
                return input_shape[0], input_shape[-1]
        else:
            return input_shape[0], input_shape[1]

    @staticmethod
    def dot_product(x, kernel):
        """
        Wrapper for dot product operation, in order to be compatible with both
        Theano and Tensorflow
        Args:
            x (): input
            kernel (): weights
        Returns:
        """
        if K.backend() == 'tensorflow':
            return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
        else:
            return K.dot(x, kernel)

In [492]:
# attention-based lstm with aspect embedding
def atae_lstm(lstm_units = 128):
    input_text = Input(shape=(Max_Len,))
    input_aspect = Input(shape=(1,),)

    word_embedding = Embedding(NUM_WORDS, EMBEDDING_SIZE, input_length=Max_Len)
    text_embed = SpatialDropout1D(0.2)(word_embedding(input_text))

    asp_embedding = Embedding(NUM_WORDS, EMBEDDING_SIZE, input_length=Max_Len)

    aspect_embed = asp_embedding(input_aspect)
    aspect_embed = Flatten()(aspect_embed)  # reshape to 2d
    repeat_aspect = RepeatVector(Max_Len)(aspect_embed)  # repeat aspect for every word in sequence

    input_concat = concatenate([text_embed, repeat_aspect], axis=-1)
    print(input_concat.shape)
    hidden_vecs, state_h, _ = LSTM(lstm_units, return_sequences=True, return_state=True)(input_concat)
    concat = concatenate([hidden_vecs, repeat_aspect], axis=-1)

    # apply attention mechanism
    attend_weight = Attention()(concat)
    attend_weight_expand = Lambda(lambda x: K.expand_dims(x))(attend_weight)
    attend_hidden = multiply([hidden_vecs, attend_weight_expand])
    attend_hidden = Lambda(lambda x: K.sum(x, axis=1))(attend_hidden)

    attend_hidden_dense = Dense(lstm_units)(attend_hidden)
    last_hidden_dense = Dense(lstm_units)(state_h)
    final_output = Activation('tanh')(add([attend_hidden_dense, last_hidden_dense]))
    output = Dense(3, activation='softmax')(final_output)
    return Model([input_text, input_aspect], output)

In [493]:

atae_lstm_model =  atae_lstm()
atae_lstm_model.summary()

(None, 67, 256)
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_11 (InputLayer)          [(None, 67)]         0           []                               
                                                                                                  
 embedding_11 (Embedding)       (None, 1, 128)       12800000    ['input_12[0][0]']               
                                                                                                  
 embedding_10 (Embedding)       (None, 67, 128)      12800000    ['input_11[0][0]']               
                                                                            

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [494]:

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                                    patience = 2,
                                                    verbose=1,
                                                    factor=0.1,
                                                    min_lr=0.000001)

atae_lstm_model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
atae_lstm_model.fit(x = train_data, y = y_train, validation_data = (val_data, y_val), batch_size=16, epochs=7, callbacks = [learning_rate_reduction])

Epoch 1/7
700/700 [==============================] - 35s 44ms/step - loss: 0.8706 - accuracy: 0.5898 - val_loss: 0.7827 - val_accuracy: 0.6269 - lr: 0.0010
Epoch 2/7
700/700 [==============================] - 11s 15ms/step - loss: 0.7187 - accuracy: 0.6810 - val_loss: 0.7965 - val_accuracy: 0.6276 - lr: 0.0010
Epoch 3/7
700/700 [==============================] - 11s 16ms/step - loss: 0.6289 - accuracy: 0.7231 - val_loss: 0.8356 - val_accuracy: 0.6291 - lr: 0.0010
Epoch 4/7
700/700 [==============================] - 11s 16ms/step - loss: 0.5567 - accuracy: 0.7511 - val_loss: 0.8454 - val_accuracy: 0.6434 - lr: 0.0010
Epoch 5/7
700/700 [==============================] - 11s 15ms/step - loss: 0.4877 - accuracy: 0.7795 - val_loss: 0.9116 - val_accuracy: 0.6246 - lr: 0.0010
Epoch 6/7
700/700 [==============================] - ETA: 0s - loss: 0.4180 - accuracy: 0.8086
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
700/700 [==============================] - 11s 1

In [495]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
42/42 [==============================] - 0s 4ms/step - loss: 1.2057 - accuracy: 0.6377
test loss, test acc: [1.205688714981079, 0.6377245783805847]
Accuracy of the model is -  63.77245783805847 %


##CROSS VALIDATION

##CROSS VALIDATE ON SEMEVAL RESTAURANT

In [496]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_SemEval2016/restaurants_test_single.csv", encoding='utf8')

In [497]:
df2_test

,sentence Id,sentence,aspect_category,aspect_term,from,to,polarity
0,en_BlueRibbonSushi_478218171:0,Yum!,FOOD#QUALITY,NaN,0,0,positive
1,en_BlueRibbonSushi_478218171:1,Serves really good sushi.,FOOD#QUALITY,sushi,19,24,positive
2,en_BlueRibbonSushi_478218171:2,Not the biggest portions but adequate.,FOOD#STYLE_OPTIONS,portions,16,24,neutral
3,en_BlueRibbonSushi_478218171:3,Green Tea creme brulee is a must!,FOOD#QUALITY,Green Tea creme brulee,0,22,positive
4,en_BlueRibbonSushi_478218171:4,Don't leave the restaurant without it.,FOOD#QUALITY,NaN,0,0,positive
...,...,...,...,...,...,...,...
854,en_Ray'sBoathouse_477775918:13,"All considered, I have to say that Ray's Boath...",RESTAURANT#GENERAL,Ray's Boathouse,35,50,positive
855,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,SERVICE#GENERAL,server,88,94,positive
856,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,FOOD#QUALITY,food,99,103,positive
857,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,RESTAURANT#MISCELLANEOUS,NaN,0,0,negative


In [498]:
df2_test.drop(columns=['from' , 'to', 'aspect_category'], inplace=True)


In [499]:
df2_test=df2_test.dropna(axis=0)


In [500]:
df2_test

,sentence Id,sentence,aspect_term,polarity
1,en_BlueRibbonSushi_478218171:1,Serves really good sushi.,sushi,positive
2,en_BlueRibbonSushi_478218171:2,Not the biggest portions but adequate.,portions,neutral
3,en_BlueRibbonSushi_478218171:3,Green Tea creme brulee is a must!,Green Tea creme brulee,positive
6,en_BlueRibbonSushi_478218345:1,– I can't say enough about this place.,place,positive
7,en_BlueRibbonSushi_478218345:2,It has great sushi and even better service.,sushi,positive
...,...,...,...,...
852,en_Ray'sBoathouse_477775918:10,Our server continued to be attentive throughou...,Ray's,neutral
854,en_Ray'sBoathouse_477775918:13,"All considered, I have to say that Ray's Boath...",Ray's Boathouse,positive
855,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,server,positive
856,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,food,positive


In [501]:
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1
df2_test['polarity'][df2_test["polarity"] == 'neutral'] = 2


<ipython-input-501-ee27ebe34892>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
<ipython-input-501-ee27ebe34892>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1
<ipython-input-501-ee27ebe34892>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'neutral'] = 2


In [502]:
X_test = df2_test

In [503]:

## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


<ipython-input-503-9dad12974081>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [504]:
y_test = X_test["polarity"]

In [505]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_term']) # list of tokenized sentences


In [506]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [507]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [508]:

print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")


Evaluate on test data
21/21 [==============================] - 0s 4ms/step - loss: 1.5563 - accuracy: 0.5308
test loss, test acc: [1.5563015937805176, 0.5307692289352417]
Accuracy of the model is -  53.07692289352417 %


##CROSS VALIDATE ON YASO

In [509]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_YASO/TSA-MD_val_single.csv")

In [510]:
def delete_rows_with_target_not_found(df):
    return df.loc[df['aspect_term'] != 'Target Not Found']


In [511]:
df2_test = delete_rows_with_target_not_found(df2_test)

In [512]:
df2_test

,sentence,aspect_term,begin,end,polarity
0,"""Typical"" Italian food was far from typical.",Italian food,10,22,negative
1,After a few weeks the tablet started asking fo...,sound,128,133,positive
2,After a few weeks the tablet started asking fo...,buy,251,254,positive
3,After a few weeks the tablet started asking fo...,wireless headphones,202,221,positive
4,After a few weeks the tablet started asking fo...,resolution of the camera is decent,88,122,positive
...,...,...,...,...,...
306,"yesterday, I ordered four chicken sandwiches b...",delivered,105,114,negative
307,"yesterday, I ordered four chicken sandwiches b...",quality of food,53,68,negative
308,"yesterday, I ordered four chicken sandwiches b...",chicken sandwiches,26,44,negative
309,you can find there anything you want .,there,13,18,positive


In [513]:
df2_test

,sentence,aspect_term,begin,end,polarity
0,"""Typical"" Italian food was far from typical.",Italian food,10,22,negative
1,After a few weeks the tablet started asking fo...,sound,128,133,positive
2,After a few weeks the tablet started asking fo...,buy,251,254,positive
3,After a few weeks the tablet started asking fo...,wireless headphones,202,221,positive
4,After a few weeks the tablet started asking fo...,resolution of the camera is decent,88,122,positive
...,...,...,...,...,...
306,"yesterday, I ordered four chicken sandwiches b...",delivered,105,114,negative
307,"yesterday, I ordered four chicken sandwiches b...",quality of food,53,68,negative
308,"yesterday, I ordered four chicken sandwiches b...",chicken sandwiches,26,44,negative
309,you can find there anything you want .,there,13,18,positive


In [514]:
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1



<ipython-input-514-5dfe2b17fbcd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1


In [515]:
X_test = df2_test

In [516]:
## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [517]:
y_test = X_test["polarity"]

In [518]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_term']) # list of tokenized sentences


In [519]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [520]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [521]:

print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")


Evaluate on test data
10/10 [==============================] - 0s 8ms/step - loss: 1.4005 - accuracy: 0.4887
test loss, test acc: [1.4005056619644165, 0.4887459874153137]
Accuracy of the model is -  48.87459874153137 %


##CROSS VALIDATE ON MAMS

In [522]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_MAMS/ATSA_test_single.csv", encoding='utf8')

In [523]:
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1
df2_test['polarity'][df2_test["polarity"] == 'neutral'] = 2

In [524]:
df2_test

,sentence,aspect_terms,polarity
0,The food was served promptly but the meal wasn...,food,2
1,The food was served promptly but the meal wasn...,served,1
2,The food was served promptly but the meal wasn...,appetizers,1
3,"When I got home, there was a message on the ma...",owner,2
4,"When I got home, there was a message on the ma...",waitress,0
...,...,...,...
1331,Pair you food with the excellent beers on tap ...,priced wine list,1
1332,"They were out of Hennessy, Alize, Mojitos, Pin...",Mojitos,2
1333,"They were out of Hennessy, Alize, Mojitos, Pin...",Pinot,2
1334,"They were out of Hennessy, Alize, Mojitos, Pin...",drink list,0


In [525]:
X_test = df2_test

In [526]:
## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [527]:
y_test = X_test["polarity"]

In [528]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_terms']) # list of tokenized sentences


In [529]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [530]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [531]:
"""
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")
"""

'\nprint("Evaluate on test data")\nresults = atae_lstm_model.evaluate(x = test_data , y = y_test )\nprint("test loss, test acc:", results)\nprint("Accuracy of the model is - " , results[1]*100 , "%")\n'

##CROSS VALIDATE ON SENTIHOOD DATASET


In [532]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_Sentihood/sentihood_test_single.csv", encoding='utf8')

In [533]:

df2_test.drop(columns=['to' , 'from'], inplace=True)

In [534]:
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1


<ipython-input-534-27e789cd0644>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
<ipython-input-534-27e789cd0644>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1


In [535]:
X_test = df2_test

In [536]:
## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [537]:
y_test = X_test["polarity"]

In [538]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_term']) # list of tokenized sentences


In [539]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [540]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [541]:

print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")


Evaluate on test data
53/53 [==============================] - 0s 7ms/step - loss: 2.6353 - accuracy: 0.4669
test loss, test acc: [2.6353414058685303, 0.4669446051120758]
Accuracy of the model is -  46.69446051120758 %


##CROSS VALIDATE ON DOTSA BOOKS

In [542]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_dotsa/new_Books_test.csv", encoding='utf8')

In [543]:
train1 = []
test1 = []
val1 = []

def dataset1(data , list1):
  for text, aspects, sentiments in zip(data['Text'], data['Aspect'], data['Sentiment']):
    aspect_list = aspects.split(', ')
    sentiment_list = sentiments.split(', ')

    for aspect, sentiment in zip(aspect_list, sentiment_list):
      if ", " in aspect:
        sub_aspects = aspect.split(", ")
        for sub_aspect in sub_aspects:
          list1.append({'Text': text, 'Aspect': sub_aspect, 'Sentiment': sentiment})
      else:
        list1.append({'Text': text, 'Aspect': aspect, 'Sentiment': sentiment})

  return list1

te1 = dataset1(df2_test , test1)

# Create a new dataframe from the expanded data

test_df = pd.DataFrame(te1)




In [544]:
## change the labels to integers for the training phase
test_df['Sentiment'][test_df["Sentiment"] == 'Negative'] = 0
test_df['Sentiment'][test_df["Sentiment"] == 'Positive'] = 1
test_df['Sentiment'][test_df["Sentiment"] == 'Mixed'] = 2



In [545]:
test_df

,Text,Aspect,Sentiment
0,This story is one of too many heartbreaking st...,story,2
1,Good Faye Kellerman ! ! ! She manages to keep ...,Faye Kellerman,1
2,Good Faye Kellerman ! ! ! She manages to keep ...,characters,1
3,Good Faye Kellerman ! ! ! She manages to keep ...,it,1
4,Good book on cancer -- a little longer than I ...,book,1
...,...,...,...
363,This was my first experience reading a Faye Ke...,it,2
364,This was my first experience reading a Faye Ke...,it--story,1
365,This was my first experience reading a Faye Ke...,it--plot,0
366,This was my first experience reading a Faye Ke...,it--characters,1


In [546]:
X_test = test_df

In [547]:
## Convert polarity Values to Integers
X_test["Sentiment"] = pd.to_numeric(X_test["Sentiment"],errors='coerce')


In [548]:
y_test = X_test["Sentiment"]

In [549]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.Text) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['Aspect']) # list of tokenized sentences


In [550]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [551]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [552]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
12/12 [==============================] - 0s 6ms/step - loss: 1.8105 - accuracy: 0.3342
test loss, test acc: [1.8105124235153198, 0.33423912525177]
Accuracy of the model is -  33.423912525177 %


##CROSS VALIDATE ON DOTSA CLOTHING

In [553]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_dotsa/new_Clothing_test.csv", encoding='utf8')

In [554]:
train1 = []
test1 = []
val1 = []

def dataset1(data , list1):
  for text, aspects, sentiments in zip(data['Text'], data['Aspect'], data['Sentiment']):
    aspect_list = aspects.split(', ')
    sentiment_list = sentiments.split(', ')

    for aspect, sentiment in zip(aspect_list, sentiment_list):
      if ", " in aspect:
        sub_aspects = aspect.split(", ")
        for sub_aspect in sub_aspects:
          list1.append({'Text': text, 'Aspect': sub_aspect, 'Sentiment': sentiment})
      else:
        list1.append({'Text': text, 'Aspect': aspect, 'Sentiment': sentiment})

  return list1

te1 = dataset1(df2_test , test1)

# Create a new dataframe from the expanded data

test_df = pd.DataFrame(te1)




In [555]:
## change the labels to integers for the training phase
test_df['Sentiment'][test_df["Sentiment"] == 'Negative'] = 0
test_df['Sentiment'][test_df["Sentiment"] == 'Positive'] = 1
test_df['Sentiment'][test_df["Sentiment"] == 'Mixed'] = 2



In [556]:
X_test = test_df

In [557]:
## Convert polarity Values to Integers
X_test["Sentiment"] = pd.to_numeric(X_test["Sentiment"],errors='coerce')


In [558]:
y_test = X_test["Sentiment"]

In [559]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.Text) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['Aspect']) # list of tokenized sentences


In [560]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [561]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [562]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
7/7 [==============================] - 0s 7ms/step - loss: 1.8444 - accuracy: 0.3673
test loss, test acc: [1.844429850578308, 0.36734694242477417]
Accuracy of the model is -  36.73469424247742 %


##CROSS VALIDATE ON DOTSA HOTELS

In [563]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_dotsa/new_Hotels_test.csv", encoding='utf8')

In [564]:
train1 = []
test1 = []
val1 = []

def dataset1(data , list1):
  for text, aspects, sentiments in zip(data['Text'], data['Aspect'], data['Sentiment']):
    aspect_list = aspects.split(', ')
    sentiment_list = sentiments.split(', ')

    for aspect, sentiment in zip(aspect_list, sentiment_list):
      if ", " in aspect:
        sub_aspects = aspect.split(", ")
        for sub_aspect in sub_aspects:
          list1.append({'Text': text, 'Aspect': sub_aspect, 'Sentiment': sentiment})
      else:
        list1.append({'Text': text, 'Aspect': aspect, 'Sentiment': sentiment})

  return list1

te1 = dataset1(df2_test , test1)

# Create a new dataframe from the expanded data

test_df = pd.DataFrame(te1)




In [565]:
## change the labels to integers for the training phase
test_df['Sentiment'][test_df["Sentiment"] == 'Negative'] = 0
test_df['Sentiment'][test_df["Sentiment"] == 'Positive'] = 1
test_df['Sentiment'][test_df["Sentiment"] == 'Mixed'] = 2



In [566]:
X_test = test_df

In [567]:
## Convert polarity Values to Integers
X_test["Sentiment"] = pd.to_numeric(X_test["Sentiment"],errors='coerce')


In [568]:
y_test = X_test["Sentiment"]

In [569]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.Text) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['Aspect']) # list of tokenized sentences


In [570]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [571]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [572]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
18/18 [==============================] - 0s 5ms/step - loss: 1.4010 - accuracy: 0.5064
test loss, test acc: [1.4009952545166016, 0.506375253200531]
Accuracy of the model is -  50.6375253200531 %


##CROSS VALIDATE ON DOTSA RESTAURANTS

In [573]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_dotsa/new_Restaurant_test.csv", encoding='utf8')

In [574]:
train1 = []
test1 = []
val1 = []

def dataset1(data , list1):
  for text, aspects, sentiments in zip(data['Text'], data['Aspect'], data['Sentiment']):
    aspect_list = aspects.split(', ')
    sentiment_list = sentiments.split(', ')

    for aspect, sentiment in zip(aspect_list, sentiment_list):
      if ", " in aspect:
        sub_aspects = aspect.split(", ")
        for sub_aspect in sub_aspects:
          list1.append({'Text': text, 'Aspect': sub_aspect, 'Sentiment': sentiment})
      else:
        list1.append({'Text': text, 'Aspect': aspect, 'Sentiment': sentiment})

  return list1

te1 = dataset1(df2_test , test1)

# Create a new dataframe from the expanded data

test_df = pd.DataFrame(te1)




In [575]:
## change the labels to integers for the training phase
test_df['Sentiment'][test_df["Sentiment"] == 'Negative'] = 0
test_df['Sentiment'][test_df["Sentiment"] == 'Positive'] = 1
test_df['Sentiment'][test_df["Sentiment"] == 'Mixed'] = 2



In [576]:
X_test = test_df

In [577]:
## Convert polarity Values to Integers
X_test["Sentiment"] = pd.to_numeric(X_test["Sentiment"],errors='coerce')


In [578]:
y_test = X_test["Sentiment"]

In [579]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.Text) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['Aspect']) # list of tokenized sentences


In [580]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [581]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [582]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
22/22 [==============================] - 0s 6ms/step - loss: 1.5788 - accuracy: 0.4560
test loss, test acc: [1.578816294670105, 0.45601174235343933]
Accuracy of the model is -  45.60117423534393 %
